In [1]:
!pwd

/_disk_misc/api


Leer archivo

Importar modulos

In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import when, col
from pyspark.sql.functions import sum
from pyspark.sql.functions import to_date
from pyspark.sql.functions import asc
import os
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'

In [3]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/16 11:44:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.conf.set("spark.sql.files.maxPartitionBytes", "1048576")


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.sql.functions import split, explode

In [6]:
data = pd.read_csv('movies_dataset.csv')

Crear una instancia de SparkSession

Crear un DataFrame de PySpark a partir de DataFrame de Pandas

In [7]:
movie = spark.createDataFrame(data)

Listar columnas

In [8]:
print(movie.columns)

['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27']


Eliminar columnas

In [9]:
columnas_a_eliminar = ["video","imdb_id","adult","original_title","vote_count","poster_path" , "homepage"]
movie = movie.drop(*columnas_a_eliminar)
movie = movie.drop('Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27')

Listar tipo de dato de cada columna

In [10]:
for col in movie.columns:
    print(col, "el tipo de dato del campo es:", movie.schema[col].dataType)

belongs_to_collection el tipo de dato del campo es: StringType()
budget el tipo de dato del campo es: StringType()
genres el tipo de dato del campo es: StringType()
id el tipo de dato del campo es: StringType()
original_language el tipo de dato del campo es: StringType()
overview el tipo de dato del campo es: StringType()
popularity el tipo de dato del campo es: StringType()
production_companies el tipo de dato del campo es: StringType()
production_countries el tipo de dato del campo es: StringType()
release_date el tipo de dato del campo es: StringType()
revenue el tipo de dato del campo es: StringType()
runtime el tipo de dato del campo es: StringType()
spoken_languages el tipo de dato del campo es: StringType()
status el tipo de dato del campo es: StringType()
tagline el tipo de dato del campo es: StringType()
title el tipo de dato del campo es: StringType()
vote_average el tipo de dato del campo es: StringType()


Cambiar tipo de datos de columnas `budget`, `revenue`, `release_date`

In [11]:
from pyspark.sql.functions import col


movie = movie.withColumn("budget", col("budget").cast("integer"))
movie = movie.withColumn("revenue", col("revenue").cast("integer"))
movie = movie.withColumn('release_date', to_date(movie.release_date))

Comprobar tipo de dato del campo `budget`, `revenue`, `release_date`

In [12]:
movie.schema["budget"].dataType

IntegerType()

In [13]:
movie.schema["revenue"].dataType

IntegerType()

In [14]:
movie.schema["release_date"].dataType

DateType()

Listar valores nulos por columna

In [15]:
from pyspark.sql.functions import col, sum

movie.select(*(sum(col(c).isNull().cast('int')).alias(c) for c in movie.columns)).pandas_api()

23/05/16 11:44:15 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/05/16 11:44:16 WARN TaskSetManager: Stage 0 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average
0,0,3,0,0,0,0,0,0,0,210,127,0,0,0,0,0,0


Reemplazar los valores nulos de las columnas budget y revenue por 0(cero).

In [16]:
movie = movie.fillna({"budget": 0})
movie = movie.fillna({"revenue": 0})

Listar valores nulos de las columnas revenue y buget

In [17]:
movie.select('revenue','budget').where(movie['budget'].isNull()).pandas_api()

,revenue,budget


Listar valores nulos por columna

In [18]:
print(movie.columns)

['belongs_to_collection', 'budget', 'genres', 'id', 'original_language', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average']


Extraer del columna `belongs_to_collection` el campo `name`

In [19]:
from pyspark.sql.functions import col
collection = movie.select(col('belongs_to_collection'))

In [20]:
from pyspark.sql.functions import from_json

# Especificar el esquema para la columna "datos" en formato JSON
json_schema = "struct<id:int,name:string,poster_path:string,backdrop_path:string>"
# Convertir la columna "datos" en un Map utilizando el esquema definido
collection = collection.withColumn("belongs_to_collection", from_json(col("belongs_to_collection"), json_schema))
# Crear nuevas columnas a partir de las claves del Map
claves = ["name"]
for clave in claves:
    collection = collection.withColumn(clave, col("belongs_to_collection").getItem(clave))

In [21]:
collection = collection.drop("belongs_to_collection")

In [22]:
collection.count()- movie.count()

23/05/16 11:44:18 WARN TaskSetManager: Stage 3 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:44:18 WARN TaskSetManager: Stage 6 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


0

Cantidad de valores nulos campo `revenue`

In [23]:
from pyspark.sql.functions import count, when, col

# Contar cantidad de valores nulos en la columna "my_column"
null_count = movie.select(count(when(col("revenue").isNull(), True)).alias("null_count")).collect()[0]["null_count"]

print("Cantidad de valores nulos en la columna 'revenue':", null_count)

23/05/16 11:44:18 WARN TaskSetManager: Stage 9 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


Cantidad de valores nulos en la columna 'revenue': 0


In [24]:
dataframe_api = movie.select(col("title"),col("release_date"),col("budget"),col("revenue"))

In [25]:
dataframe_api.limit(3).pandas_api()

23/05/16 11:44:19 WARN TaskSetManager: Stage 12 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,title,release_date,budget,revenue
0,Toy Story,1995-10-30,30000000,373554033
1,Jumanji,1995-12-15,65000000,262797249
2,Grumpier Old Men,1995-12-22,0,0


In [26]:
from pyspark.sql.functions import col

dataframe_api = dataframe_api.withColumn("profit", col("revenue") - col("budget"))

In [27]:
from pyspark.sql.functions import expr

dataframe_api = dataframe_api.withColumn('return', expr('revenue / budget'))

In [28]:
dataframe_api.limit(3).pandas_api()

23/05/16 11:44:19 WARN TaskSetManager: Stage 15 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,title,release_date,budget,revenue,profit,return
0,Toy Story,1995-10-30,30000000,373554033,343554033,12.451801
1,Jumanji,1995-12-15,65000000,262797249,197797249,4.043035
2,Grumpier Old Men,1995-12-22,0,0,0,NaN


In [29]:
from pyspark.sql.functions import to_date, date_format
import pyspark.sql.functions as F

#df = df.withColumn('dia_semana', F.date_format(df.fecha, 'EEEE', 'es_ES'))

Formatear `release_date` en campos `year` `month` y `day_week`

In [30]:
dataframe_api = dataframe_api.withColumn('year', date_format(dataframe_api.release_date, 'yyyy')) \
       .withColumn('month', date_format(dataframe_api.release_date, 'MMMM')) \
       .withColumn('day', date_format(dataframe_api.release_date, 'dd')) \
       .withColumn('day_week', F.date_format(dataframe_api.release_date, 'EEEE'))

Eliminar campo `release_date`

In [31]:
dataframe_api = dataframe_api.drop("release_date")

In [32]:
dataframe_api.limit(3).pandas_api()

23/05/16 11:44:20 WARN TaskSetManager: Stage 18 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,title,budget,revenue,profit,return,year,month,day,day_week
0,Toy Story,30000000,373554033,343554033,12.451801,1995,October,30,Monday
1,Jumanji,65000000,262797249,197797249,4.043035,1995,December,15,Friday
2,Grumpier Old Men,0,0,0,NaN,1995,December,22,Friday


Reemplazar days de la semana y meses del año en ingles por catellano

In [33]:
#dia de la semana
dataframe_api = dataframe_api.withColumn('day_week', when(col('day_week') == "Monday", "Lunes").otherwise(col('day_week')))
dataframe_api = dataframe_api.withColumn('day_week', when(col('day_week') == "Tuesday", "Martes").otherwise(col('day_week')))
dataframe_api = dataframe_api.withColumn('day_week', when(col('day_week') == "Wednesday", "Miercoles").otherwise(col('day_week')))
dataframe_api = dataframe_api.withColumn('day_week', when(col('day_week') == "Thursday", "Jueves").otherwise(col('day_week')))
dataframe_api = dataframe_api.withColumn('day_week', when(col('day_week') == "Friday", "Viernes").otherwise(col('day_week')))
dataframe_api = dataframe_api.withColumn('day_week', when(col('day_week') == "Saturday", "Sabado").otherwise(col('day_week')))
dataframe_api = dataframe_api.withColumn('day_week', when(col('day_week') == "Sunday", "Domingo").otherwise(col('day_week')))
#mes del año
dataframe_api = dataframe_api.withColumn('month', when(col('month') == "January", "Enero").otherwise(col('month')))
dataframe_api = dataframe_api.withColumn('month', when(col('month') == "February", "Febrero").otherwise(col('month')))
dataframe_api = dataframe_api.withColumn('month', when(col('month') == "March", "Marzo").otherwise(col('month')))
dataframe_api = dataframe_api.withColumn('month', when(col('month') == "April", "Abril").otherwise(col('month')))
dataframe_api = dataframe_api.withColumn('month', when(col('month') == "May", "Mayo").otherwise(col('month')))
dataframe_api = dataframe_api.withColumn('month', when(col('month') == "June", "Junio").otherwise(col('month')))
dataframe_api = dataframe_api.withColumn('month', when(col('month') == "July", "Julio").otherwise(col('month')))
dataframe_api = dataframe_api.withColumn('month', when(col('month') == "August", "Agosto").otherwise(col('month')))
dataframe_api = dataframe_api.withColumn('month', when(col('month') == "September", "Septiembre").otherwise(col('month')))
dataframe_api = dataframe_api.withColumn('month', when(col('month') == "October", "Octubre").otherwise(col('month')))
dataframe_api = dataframe_api.withColumn('month', when(col('month') == "November", "Noviembre").otherwise(col('month')))
dataframe_api = dataframe_api.withColumn('month', when(col('month') == "December", "Diciembre").otherwise(col('month')))

In [34]:
dataframe_api.limit(3).pandas_api()

23/05/16 11:44:21 WARN TaskSetManager: Stage 21 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,title,budget,revenue,profit,return,year,month,day,day_week
0,Toy Story,30000000,373554033,343554033,12.451801,1995,Octubre,30,Lunes
1,Jumanji,65000000,262797249,197797249,4.043035,1995,Diciembre,15,Viernes
2,Grumpier Old Men,0,0,0,NaN,1995,Diciembre,22,Viernes


In [35]:
dataframe_api

DataFrame[title: string, budget: int, revenue: int, profit: int, return: double, year: string, month: string, day: string, day_week: string]

In [36]:
from pyspark.sql.functions import monotonically_increasing_id

Crear una columna `key` autoincremetal en Dataframe `dataframe_api`

In [37]:
dataframe_api = dataframe_api.withColumn('key', monotonically_increasing_id())

In [38]:
dataframe_api.limit(3).pandas_api()

23/05/16 11:44:21 WARN TaskSetManager: Stage 24 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,title,budget,revenue,profit,return,year,month,day,day_week,key
0,Toy Story,30000000,373554033,343554033,12.451801,1995,Octubre,30,Lunes,0
1,Jumanji,65000000,262797249,197797249,4.043035,1995,Diciembre,15,Viernes,1
2,Grumpier Old Men,0,0,0,NaN,1995,Diciembre,22,Viernes,2


Crear una columna `key` autoincremetal en Dataframe `collection`

In [39]:

collection = collection.withColumn('key', monotonically_increasing_id())
collection.limit(3).pandas_api()

23/05/16 11:44:22 WARN TaskSetManager: Stage 27 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,name,key
0,Toy Story Collection,0
1,None,1
2,Grumpy Old Men Collection,2


Unir los dataframonth `dataframe_api` y `collection`utilizando la clave `key`

In [40]:
dataframe_api = dataframe_api.join(collection, 'key')

In [41]:
dataframe_api.limit(3).pandas_api()

23/05/16 11:44:22 WARN TaskSetManager: Stage 30 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:44:23 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
23/05/16 11:44:23 WARN TaskSetManager: Stage 31 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,key,title,budget,revenue,profit,return,year,month,day,day_week,name
0,26,Now and Then,12000000,27400000,15400000,2.283333,1995,Octubre,20,Viernes,None
1,29,Shanghai Triad,0,0,0,NaN,1995,Abril,30,Domingo,None
2,474,Judgment Night,0,12136938,12136938,NaN,1993,Octubre,15,Viernes,None


Cambiar nombre campo `name` por `collection`

In [42]:
dataframe_api = dataframe_api.withColumnRenamed("name", "collection")

In [43]:
dataframe_api.limit(3).pandas_api()

23/05/16 11:44:24 WARN TaskSetManager: Stage 39 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:44:24 WARN TaskSetManager: Stage 40 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,key,title,budget,revenue,profit,return,year,month,day,day_week,collection
0,26,Now and Then,12000000,27400000,15400000,2.283333,1995,Octubre,20,Viernes,None
1,29,Shanghai Triad,0,0,0,NaN,1995,Abril,30,Domingo,None
2,474,Judgment Night,0,12136938,12136938,NaN,1993,Octubre,15,Viernes,None


In [44]:
dataframe_api.count()

23/05/16 11:44:25 WARN TaskSetManager: Stage 48 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


45466

In [45]:
dataframe_api.coalesce(1).write.format("csv").mode('overwrite').option("header", "true").save("csv")

23/05/16 11:44:26 WARN TaskSetManager: Stage 56 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:44:26 WARN TaskSetManager: Stage 57 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


In [46]:
path = "csv"

In [47]:
data_00 = spark.read.option("header", "true").option("header", "true").option("inferSchema", "true").csv(path)

In [48]:


data_00 = data_00.orderBy(asc("key"))

In [49]:
dataframe_api.limit(3).pandas_api()

23/05/16 11:44:28 WARN TaskSetManager: Stage 64 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:44:28 WARN TaskSetManager: Stage 65 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,key,title,budget,revenue,profit,return,year,month,day,day_week,collection
0,26,Now and Then,12000000,27400000,15400000,2.283333,1995,Octubre,20,Viernes,None
1,29,Shanghai Triad,0,0,0,NaN,1995,Abril,30,Domingo,None
2,474,Judgment Night,0,12136938,12136938,NaN,1993,Octubre,15,Viernes,None


In [50]:
country =  movie.select(col("production_countries"))

In [51]:
country.limit(3).pandas_api()

23/05/16 11:44:29 WARN TaskSetManager: Stage 73 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,production_countries
0,"[{'iso_3166_1': 'US', 'name': 'United States o..."
1,"[{'iso_3166_1': 'US', 'name': 'United States o..."
2,"[{'iso_3166_1': 'US', 'name': 'United States o..."


In [52]:
data_country = spark.createDataFrame(data)

In [53]:
country =  movie.select(col("production_countries"))

In [54]:
country.limit(3).pandas_api()

23/05/16 11:44:34 WARN TaskSetManager: Stage 76 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,production_countries
0,"[{'iso_3166_1': 'US', 'name': 'United States o..."
1,"[{'iso_3166_1': 'US', 'name': 'United States o..."
2,"[{'iso_3166_1': 'US', 'name': 'United States o..."


In [55]:
from pyspark.sql.functions import from_json, col

# Define el esquema de datos JSON
json_schema_country = "array<struct<iso_3166_1:string, name:string>>"

# Convierte la columna de JSON a una estructura de Spark
country = country.withColumn("datos_struct", from_json(col("production_countries"), json_schema_country))

In [56]:
country.limit(3).pandas_api()

23/05/16 11:44:34 WARN TaskSetManager: Stage 79 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,production_countries,datos_struct
0,"[{'iso_3166_1': 'US', 'name': 'United States o...","[(US, United States of America)]"
1,"[{'iso_3166_1': 'US', 'name': 'United States o...","[(US, United States of America)]"
2,"[{'iso_3166_1': 'US', 'name': 'United States o...","[(US, United States of America)]"


In [57]:
# Crea una columna para cada elemento de la lista
columns_country = country.selectExpr("max(size(datos_struct)) as max_cols").collect()[0]["max_cols"]

23/05/16 11:44:35 WARN TaskSetManager: Stage 82 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


In [58]:
for i in range(columns_country):
    country = country.withColumn(f"name_{i+1}", col("datos_struct")[i].getField("name"))

In [59]:
country = country.drop("production_countries","datos_struct")

In [60]:
from pyspark.sql.functions import concat_ws

country = country.select(concat_ws("-", *country.columns).alias("country"))

In [61]:
country.limit(20).pandas_api()

23/05/16 11:44:36 WARN TaskSetManager: Stage 85 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,country
0,United States of America
1,United States of America
2,United States of America
3,United States of America
4,United States of America
5,United States of America
6,Germany-United States of America
7,United States of America
8,United States of America
9,United Kingdom-United States of America


Crear una columna `key` autoincremetal en Dataframe `country`

In [62]:
country = country.withColumn('key', monotonically_increasing_id())

In [63]:
country = country.select(col("key"), explode(split(col("country"), "-")).alias("country"))


In [64]:
country.limit(10).pandas_api()

23/05/16 11:44:36 WARN TaskSetManager: Stage 88 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,key,country
0,0,United States of America
1,1,United States of America
2,2,United States of America
3,3,United States of America
4,4,United States of America
5,5,United States of America
6,6,Germany
7,6,United States of America
8,7,United States of America
9,8,United States of America


Nota: el `key` seria el identificador de pelicula como se puede ver en el caso del key 6, donde hay dos paises

In [65]:
data_production = spark.createDataFrame(data)

In [66]:
production =  data_production.select(col("production_companies"))

In [67]:
production.limit(3).pandas_api()

23/05/16 11:44:41 WARN TaskSetManager: Stage 91 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,production_companies
0,"[{'name': 'Pixar Animation Studios', 'id': 3}]"
1,"[{'name': 'TriStar Pictures', 'id': 559}, {'na..."
2,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'..."


In [68]:
from pyspark.sql.functions import from_json, col

# Define el esquema de datos JSON
json_schema_production = "array<struct<name:string, id:int>>"

# Convierte la columna de JSON a una estructura de Spark
production = production.withColumn("datos_struct", from_json(col("production_companies"), json_schema_production))

In [69]:
production.limit(3).pandas_api()

23/05/16 11:44:41 WARN TaskSetManager: Stage 94 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,production_companies,datos_struct
0,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[(Pixar Animation Studios, 3)]"
1,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[(TriStar Pictures, 559), (Teitler Film, 2550)..."
2,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[(Warner Bros., 6194), (Lancaster Gate, 19464)]"


In [70]:
# Crea una columna para cada elemento de la lista
num_cols = production.selectExpr("max(size(datos_struct)) as max_cols").collect()[0]["max_cols"]

23/05/16 11:44:41 WARN TaskSetManager: Stage 97 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


In [71]:
for i in range(num_cols):
    production = production.withColumn(f"name_{i+1}", col("datos_struct")[i].getField("name"))

In [72]:
production = production.drop("production_companies","datos_struct")

In [73]:
from pyspark.sql.functions import concat_ws

production = production.select(concat_ws("-", *production.columns).alias("production_companies"))

In [74]:
production.limit(3).pandas_api()

23/05/16 11:44:42 WARN TaskSetManager: Stage 100 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,production_companies
0,Pixar Animation Studios
1,TriStar Pictures-Teitler Film-Interscope Commu...
2,Warner Bros.-Lancaster Gate


In [75]:
from pyspark.sql.functions import monotonically_increasing_id

Crear una columna `key` autoincremetal en Dataframe `dataframe_api`

In [76]:
production = production.withColumn('key', monotonically_increasing_id())

In [77]:
production.limit(3).pandas_api()

23/05/16 11:44:42 WARN TaskSetManager: Stage 103 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,production_companies,key
0,Pixar Animation Studios,0
1,TriStar Pictures-Teitler Film-Interscope Commu...,1
2,Warner Bros.-Lancaster Gate,2


In [78]:
from pyspark.sql.functions import split, explode

# Separar los países por guion y convertir la columna en varias filas

production = production.select(col("key"), explode(split(col("production_companies"), "-")).alias("companies"))

In [79]:
production.limit(5).pandas_api()

23/05/16 11:44:43 WARN TaskSetManager: Stage 106 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,key,companies
0,0,Pixar Animation Studios
1,1,TriStar Pictures
2,1,Teitler Film
3,1,Interscope Communications
4,2,Warner Bros.


Unir dataframe `dataframe_api` y `production`

In [80]:
production = dataframe_api.join(production, 'key', 'inner')

In [81]:
production = production.select(col("key"), col("companies"), col("profit"))

In [82]:
#production.limit(5).pandas_api()
production.orderBy(asc("key")).limit(5).pandas_api()

23/05/16 11:44:43 WARN TaskSetManager: Stage 109 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:44:44 WARN TaskSetManager: Stage 110 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:44:44 WARN TaskSetManager: Stage 111 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,key,companies,profit
0,0,Pixar Animation Studios,343554033
1,1,Teitler Film,197797249
2,1,Interscope Communications,197797249
3,1,TriStar Pictures,197797249
4,2,Lancaster Gate,0


In [83]:
production.count()

23/05/16 11:44:45 WARN TaskSetManager: Stage 118 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:44:45 WARN TaskSetManager: Stage 119 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


86659

In [84]:
data_ML = spark.createDataFrame(data)

In [85]:
genres = data_ML.select(col("genres"))

In [86]:
genres.limit(3).pandas_api()

23/05/16 11:44:50 WARN TaskSetManager: Stage 129 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,genres
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."


In [87]:
from pyspark.sql.functions import from_json, col

# Define el esquema de datos JSON
json_schema = "array<struct<id:int, name:string>>"

# Convierte la columna de JSON a una estructura de Spark
genres = genres.withColumn("datos_struct", from_json(col("genres"), json_schema))

In [88]:
genres.limit(3).pandas_api()

23/05/16 11:44:51 WARN TaskSetManager: Stage 132 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,genres,datos_struct
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[(16, Animation), (35, Comedy), (10751, Family)]"
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[(12, Adventure), (14, Fantasy), (10751, Family)]"
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[(10749, Romance), (35, Comedy)]"


In [89]:
# Crea una columna para cada elemento de la lista
num_cols = genres.selectExpr("max(size(datos_struct)) as max_cols").collect()[0]["max_cols"]
for i in range(num_cols):
    genres = genres.withColumn(f"name_{i+1}", col("datos_struct")[i].getField("name"))

23/05/16 11:44:51 WARN TaskSetManager: Stage 135 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


In [90]:
genres = genres.drop("genres","datos_struct")

In [91]:
from pyspark.sql.functions import concat_ws

genres = genres.select(concat_ws("-", *genres.columns).alias("genres"))



In [92]:
genres.limit(3).pandas_api()

23/05/16 11:44:52 WARN TaskSetManager: Stage 138 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,genres
0,Animation-Comedy-Family
1,Adventure-Fantasy-Family
2,Romance-Comedy


In [93]:
genres.count()

23/05/16 11:44:52 WARN TaskSetManager: Stage 141 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


45466

In [94]:
from pyspark.sql.functions import monotonically_increasing_id

Crear una columna `key` autoincremetal en Dataframe `genres`

In [95]:
genres = genres.withColumn('key', monotonically_increasing_id())

In [96]:
genres.limit(3).pandas_api()

23/05/16 11:44:52 WARN TaskSetManager: Stage 144 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,genres,key
0,Animation-Comedy-Family,0
1,Adventure-Fantasy-Family,1
2,Romance-Comedy,2


In [97]:
genres

DataFrame[genres: string, key: bigint]

In [98]:
#movie.limit(4).pandas_api()

In [99]:
data_genres = spark.createDataFrame(data)

In [100]:
title = data_genres.select(col("original_title"), col("vote_average"),col("vote_count"))

In [101]:
title.limit(4).pandas_api()

23/05/16 11:44:57 WARN TaskSetManager: Stage 147 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,original_title,vote_average,vote_count
0,Toy Story,7.7,5415
1,Jumanji,6.9,2413
2,Grumpier Old Men,6.5,92
3,Waiting to Exhale,6.1,34


Crear una columna `key` autoincremetal en Dataframe `collection`

In [102]:

title = title.withColumn('key', monotonically_increasing_id())


In [103]:
title.limit(4).pandas_api()

23/05/16 11:44:57 WARN TaskSetManager: Stage 150 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,original_title,vote_average,vote_count,key
0,Toy Story,7.7,5415,0
1,Jumanji,6.9,2413,1
2,Grumpier Old Men,6.5,92,2
3,Waiting to Exhale,6.1,34,3


In [104]:
from pyspark.sql.functions import col

#joined_df = df1.join(df2, col("df1.id") == col("df2.id"), "inner")

In [105]:
ML = genres.join(title, "key", "inner")


In [106]:
ML.limit(4).pandas_api()

23/05/16 11:44:58 WARN TaskSetManager: Stage 153 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:44:58 WARN TaskSetManager: Stage 154 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,key,genres,original_title,vote_average,vote_count
0,26,Comedy-Drama-Family,Now and Then,6.6,91
1,29,Drama-Crime,摇啊摇，摇到外婆桥,6.5,17
2,474,Action-Thriller-Crime,Judgment Night,6.4,79
3,964,Drama-Thriller,The Rich Man's Wife,4.9,11


In [107]:
#ML = ML.drop("key")

In [108]:
ML.limit(4).pandas_api()

23/05/16 11:44:59 WARN TaskSetManager: Stage 162 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:44:59 WARN TaskSetManager: Stage 163 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,key,genres,original_title,vote_average,vote_count
0,26,Comedy-Drama-Family,Now and Then,6.6,91
1,29,Drama-Crime,摇啊摇，摇到外婆桥,6.5,17
2,474,Action-Thriller-Crime,Judgment Night,6.4,79
3,964,Drama-Thriller,The Rich Man's Wife,4.9,11


In [109]:
#from pyspark.sql.functions import split
#ML = ML.withColumn('genres', split(ML['genres'], '-'))

In [110]:
ML.limit(4).pandas_api()

23/05/16 11:45:00 WARN TaskSetManager: Stage 171 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:45:00 WARN TaskSetManager: Stage 172 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,key,genres,original_title,vote_average,vote_count
0,26,Comedy-Drama-Family,Now and Then,6.6,91
1,29,Drama-Crime,摇啊摇，摇到外婆桥,6.5,17
2,474,Action-Thriller-Crime,Judgment Night,6.4,79
3,964,Drama-Thriller,The Rich Man's Wife,4.9,11


escribir los archivos que van a ser usados en la API

In [111]:
dataframe_api.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save("movie")

23/05/16 11:45:01 WARN TaskSetManager: Stage 180 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:45:02 WARN TaskSetManager: Stage 181 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


In [112]:
production.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save("production")

23/05/16 11:45:03 WARN TaskSetManager: Stage 186 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:45:03 WARN TaskSetManager: Stage 187 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:45:03 WARN TaskSetManager: Stage 188 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


In [113]:
country.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save("country")

23/05/16 11:45:04 WARN TaskSetManager: Stage 195 contains a task of very large size (31165 KiB). The maximum recommended task size is 1000 KiB.


In [114]:
ML.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save("ML")

23/05/16 11:45:06 WARN TaskSetManager: Stage 196 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:45:06 WARN TaskSetManager: Stage 197 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


In [115]:
country.count()

23/05/16 11:45:07 WARN TaskSetManager: Stage 202 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


55766

In [116]:
production.count()

23/05/16 11:45:07 WARN TaskSetManager: Stage 205 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.
23/05/16 11:45:08 WARN TaskSetManager: Stage 206 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


86659

In [117]:
movie.count()

23/05/16 11:45:08 WARN TaskSetManager: Stage 216 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


45466

In [118]:
recomendation =  spark.createDataFrame(data)

In [119]:
recomendation = recomendation.select(col("title"),col("overview"))

In [120]:
recomendation.limit(3).pandas_api()

23/05/16 11:45:13 WARN TaskSetManager: Stage 219 contains a task of very large size (2218 KiB). The maximum recommended task size is 1000 KiB.


,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...


In [121]:
recomendation.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save("recomendation")

23/05/16 11:45:13 WARN TaskSetManager: Stage 222 contains a task of very large size (31165 KiB). The maximum recommended task size is 1000 KiB.
